In [76]:
import sys
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import svm
from sklearn.ensemble import IsolationForest
from sklearn.naive_bayes import MultinomialNB
#from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.svm import OneClassSVM
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report


df = pd.read_csv('C:/Users/Anupama/Documents/4thsem/THESIS/OutputThreshold/Requirement_model3_3.csv',sep=',')
print("Data basic description:")
print(df.describe())
print("Data column")
print(df.columns)

Data basic description:
        Unnamed: 0  requirement_id      clarity  creativity_score  \
count  2599.000000     2599.000000  2599.000000       2599.000000   
mean   1299.000000     1513.084648    -0.882262         -0.729896   
std     750.410998      850.784110     0.470849          0.683690   
min       0.000000       11.000000    -1.000000         -1.000000   
25%     649.500000      787.500000    -1.000000         -1.000000   
50%    1299.000000     1510.000000    -1.000000         -1.000000   
75%    1948.500000     2256.500000    -1.000000         -1.000000   
max    2598.000000     2965.000000     1.000000          1.000000   

           novelty   usefulness  
count  2599.000000  2599.000000  
mean     -0.488265    -0.783763  
std       0.872864     0.621180  
min      -1.000000    -1.000000  
25%      -1.000000    -1.000000  
50%      -1.000000    -1.000000  
75%       1.000000    -1.000000  
max       1.000000     1.000000  
Data column
Index(['Unnamed: 0', 'requirement_id

In [77]:
print("Number of novel requirements:", len(df[df.novelty==-1]))
print("Number of non-novel requirements:", len(df[df.novelty==1]))
print("Number of novel clarity scores:", len(df[df.clarity==-1]))
print("Number of non-novel clarity scores:", len(df[df.clarity==1]))
print("Number of novel creativity scores:", len(df[df.creativity_score==-1]))
print("Number of non-novel creativity scores:", len(df[df.creativity_score==1]))
print("Number of novel usefulness scores:", len(df[df.usefulness==-1]))
print("Number of non-novel usefulness scores:", len(df[df.usefulness==1]))

df_requirement = df["requirement"]
df_novelty = df["novelty"]
df_creativity = df["creativity_score"]
df_clarity = df["clarity"]
df_usefulness = df["usefulness"]
df_reqOriginal = df["originalRequirement"]

Number of novel requirements: 1934
Number of non-novel requirements: 665
Number of novel clarity scores: 2446
Number of non-novel clarity scores: 153
Number of novel creativity scores: 2248
Number of non-novel creativity scores: 351
Number of novel usefulness scores: 2318
Number of non-novel usefulness scores: 281


In [78]:
#### Splitting data to training and testing dataset
requirement_train, requirement_test, novelty_train, novelty_test = train_test_split(df_requirement,df_novelty,test_size=0.2,random_state=4)
requirement_train, requirement_test, clarity_train, clarity_test = train_test_split(df_requirement,df_clarity,test_size=0.2,random_state=4)
requirement_train, requirement_test, creativity_train, creativity_test = train_test_split(df_requirement,df_creativity,test_size=0.2,random_state=4)
requirement_train, requirement_test, usefulness_train, usefulness_test = train_test_split(df_requirement,df_usefulness,test_size=0.2,random_state=4)

print("################################")

print("Length of training data:",len(requirement_train))
print("Length of test data:",len(requirement_test))
print("################################")

print("Train data stats: Total: ",len(requirement_train))
print("----------------------")

print("1. Regular novelty values:",len(requirement_train[df.novelty==1]))
print("1. Novel values:",len(requirement_train[df.novelty==-1]))

print("2. Regular clarity values:",len(requirement_train[df.clarity==1]))
print("2. Novel clarity values:",len(requirement_train[df.clarity==-1]))

print("3. Regular creativity scores:",len(requirement_train[df.creativity_score==1]))
print("3. Novel creativity scores:",len(requirement_train[df.creativity_score==-1]))

print("4. Regular usefulness values:",len(requirement_train[df.usefulness==1]))
print("4. Novel usefulness values:",len(requirement_train[df.usefulness==-1]))


print("################################")
print("----------------------")

print("Test data stats: Total: ",len(requirement_test))

print("1. Regular novelty values:",len(requirement_test[df.novelty==1]))
print("1. Novel values:",len(requirement_test[df.novelty==-1]))

print("2. Regular clarity values:",len(requirement_test[df.clarity==1]))
print("2. Novel clarity values:",len(requirement_test[df.clarity==-1]))

print("3. Regular creativity scores:",len(requirement_test[df.creativity_score==1]))
print("3. Novel creativity scores:",len(requirement_test[df.creativity_score==-1]))

print("4. Regular usefulness values:",len(requirement_test[df.usefulness==1]))
print("4. Novel usefulness values:",len(requirement_test[df.usefulness==-1]))



print(df_requirement.value_counts())



################################
Length of training data: 2079
Length of test data: 520
################################
Train data stats: Total:  2079
----------------------
1. Regular novelty values: 517
1. Novel values: 1562
2. Regular clarity values: 129
2. Novel clarity values: 1950
3. Regular creativity scores: 275
3. Novel creativity scores: 1804
4. Regular usefulness values: 229
4. Novel usefulness values: 1850
################################
----------------------
Test data stats: Total:  520
1. Regular novelty values: 148
1. Novel values: 372
2. Regular clarity values: 24
2. Novel clarity values: 496
3. Regular creativity scores: 76
3. Novel creativity scores: 444
4. Regular usefulness values: 52
4. Novel usefulness values: 468
lock                                                                                           5
thermostat                                                                                     5
automat light                                            

In [79]:
cv = CountVectorizer()
regularNoveltySet = requirement_train[df.novelty==1]
novelNovelSet = requirement_train[df.novelty==-1]
regularClaritySet = requirement_train[df.clarity==1]
novelClaritySet = requirement_train[df.clarity==-1]
regularCreativeSet = requirement_train[df.creativity_score==1]
novelCreativeSet = requirement_train[df.creativity_score==-1]
regularUsefulSet = requirement_train[df.usefulness==1]
novelUsefulSet = requirement_train[df.usefulness==-1]


requirementTrain = cv.fit_transform(requirement_train.values.astype('U'))
requirementTest = cv.fit_transform(requirement_test.values.astype('U'))

regularNoveltySet_converted = cv.transform(regularNoveltySet.values.astype('U'))
NovelNovelSet_converted = cv.transform(novelNovelSet.values.astype('U')) 

regularClaritySet_converted = cv.transform(regularClaritySet.values.astype('U'))
NovelClaritySet_converted = cv.transform(novelClaritySet.values.astype('U')) 

regularCreativeSet_converted = cv.transform(regularCreativeSet.values.astype('U'))
NovelCreativeSet_converted = cv.transform(novelCreativeSet.values.astype('U')) 

regularUsefulSet_converted = cv.transform(regularUsefulSet.values.astype('U'))
NovelUsefulSet_converted = cv.transform(novelUsefulSet.values.astype('U')) 



In [80]:
############ Isolation Forest

print("############# Novelty")

clf = IsolationForest(max_samples="auto")
clf.fit(regularNoveltySet_converted)
predictedResults = clf.predict(requirementTest)
actualResults = np.array(novelty_test)

predictResult = pd.DataFrame(data=actualResults)
#np.savetxt("outputPredicted.txt",predictResult)

result_IsolationForest = classification_report(actualResults,predictedResults)
print("Classification report for Isolation Forest:")
print(result_IsolationForest)

# Confusion matrix novelty
cnf_matrix = confusion_matrix(novelty_test,predictedResults)
tn, fp, fn, tp = confusion_matrix(novelty_test,predictedResults).ravel()
#print("tn:",tn,"fp:",fp, "fn:",fn, "tp:",tp )
print("Confusion Matrix novelty:")
print(cnf_matrix)



print("############# Clarity")
clf = IsolationForest(max_samples="auto")
clf.fit(regularClaritySet_converted)
predictedResults = clf.predict(requirementTest)
actualResults = np.array(clarity_test)


predictResult = pd.DataFrame(data=actualResults)
#np.savetxt("outputPredicted.txt",predictResult)

result_IsolationForest = classification_report(actualResults,predictedResults)
print("Classification report for Isolation Forest:")
print(result_IsolationForest)

# Confusion matrix
cnf_matrix = confusion_matrix(novelty_test,predictedResults)
tn, fp, fn, tp = confusion_matrix(novelty_test,predictedResults).ravel()
#print("tn:",tn,"fp:",fp, "fn:",fn, "tp:",tp )
print("Confusion Matrix novelty:")
print(cnf_matrix)


print("############# Creativity")

clf = IsolationForest(max_samples="auto")
clf.fit(regularCreativeSet_converted)
predictedResults = clf.predict(requirementTest)
actualResults = np.array(creativity_test)


predictResult = pd.DataFrame(data=actualResults)
#np.savetxt("outputPredicted.txt",predictResult)

result_IsolationForest = classification_report(actualResults,predictedResults)
print("Classification report for Isolation Forest:")
print(result_IsolationForest)

# Confusion matrix
cnf_matrix = confusion_matrix(novelty_test,predictedResults)
tn, fp, fn, tp = confusion_matrix(novelty_test,predictedResults).ravel()
#print("tn:",tn,"fp:",fp, "fn:",fn, "tp:",tp )
print("Confusion Matrix novelty:")
print(cnf_matrix)


print("############# Usefulness")
clf = IsolationForest(max_samples="auto")
clf.fit(regularUsefulSet_converted)
predictedResults = clf.predict(requirementTest)
actualResults = np.array(usefulness_test)


predictResult = pd.DataFrame(data=actualResults)
#np.savetxt("outputPredicted.txt",predictResult)

result_IsolationForest = classification_report(actualResults,predictedResults)
print("Classification report for Isolation Forest:")
print(result_IsolationForest)

# Confusion matrix
cnf_matrix = confusion_matrix(novelty_test,predictedResults)
tn, fp, fn, tp = confusion_matrix(usefulness_test,predictedResults).ravel()
#print("tn:",tn,"fp:",fp, "fn:",fn, "tp:",tp )
print("Confusion Matrix novelty:")
print(cnf_matrix)


# print(requirement_test)
# print(type(requirement_test))
# print(predictedResults)



############# Novelty
Classification report for Isolation Forest:
             precision    recall  f1-score   support

         -1       0.72      0.07      0.13       372
          1       0.29      0.93      0.44       148

avg / total       0.60      0.32      0.22       520

Confusion Matrix novelty:
[[ 26 346]
 [ 10 138]]
############# Clarity
Classification report for Isolation Forest:
             precision    recall  f1-score   support

         -1       0.78      0.01      0.03       496
          1       0.04      0.92      0.08        24

avg / total       0.74      0.06      0.03       520

Confusion Matrix novelty:
[[  7 365]
 [  2 146]]
############# Creativity
Classification report for Isolation Forest:
             precision    recall  f1-score   support

         -1       0.90      0.06      0.11       444
          1       0.15      0.96      0.26        76

avg / total       0.79      0.19      0.13       520

Confusion Matrix novelty:
[[ 20 352]
 [ 10 138]]
#######

In [83]:
print("#########################Novelty")

clf = svm.OneClassSVM(nu=0.1,kernel="poly",gamma=0.1)
clf.fit(regularNoveltySet_converted)
svmoneclass = clf.predict(requirementTest)
#print("Predicted values:")
#np.savetxt("outputPredicted.txt",svmoneclass)
#print(svmoneclass)
reqTestDf = pd.DataFrame(data=requirement_test)
#print(reqTestDf)
svmresultDf = pd.DataFrame(data=svmoneclass)
actualResults = np.array(novelty_test)
actualResultsDf = pd.DataFrame(data=actualResults)
print("Actual results:")
#np.savetxt("actualResults.txt",actualResultsDf)
# print(actualResultsDf)
finalResult=pd.DataFrame()
finalResult["requirement"] = reqTestDf
finalResult["PredictionsSVM"] = svmresultDf
finalResult["GroundTruth"] = actualResultsDf

finalResult.to_csv('C:/Users/Anupama/Documents/4thsem/THESIS/Output/Resultsvm.csv',encoding='utf-8')

##result = pd.concat([reqTestDf,svmresultDf,actualResultsDf],ignore_index=True)
#print(result)
print("Classification report for SVM One class")
result = classification_report(actualResults,svmoneclass)
print(result)

cnf_matrix = confusion_matrix(novelty_test,svmoneclass)
tp, fn, fp, tn = confusion_matrix(novelty_test,svmoneclass).ravel()
print("tp:",tp,"fn:",fn, "fp:",fp, "tn:",tn )
print("Confusion Matrix")
print(cnf_matrix)


print("############## Clarity")
clf = svm.OneClassSVM(nu=0.1,kernel="poly",gamma=0.1)
clf.fit(regularClaritySet_converted)
svmoneclass = clf.predict(requirementTest)
actualResults = np.array(clarity_test)
print("Actual results:")

print("Classification report for SVM One class")
result = classification_report(actualResults,svmoneclass)
print(result)

cnf_matrix = confusion_matrix(novelty_test,svmoneclass)
tp, fn, fp, tn = confusion_matrix(clarity_test,svmoneclass).ravel()
print("tp:",tp,"fn:",fn, "fp:",fp, "tn:",tn )
print("Confusion Matrix")
print(cnf_matrix)

print("############## Creativity")
clf = svm.OneClassSVM(nu=0.1,kernel="poly",gamma=0.1)
clf.fit(regularCreativeSet_converted)
svmoneclass = clf.predict(requirementTest)
actualResults = np.array(creativity_test)
print("Actual results:")

print("Classification report for SVM One class")
result = classification_report(actualResults,svmoneclass)
print(result)

cnf_matrix = confusion_matrix(novelty_test,svmoneclass)
tp, fn, fp, tn = confusion_matrix(creativity_test,svmoneclass).ravel()
print("tp:",tp,"fn:",fn, "fp:",fp, "tn:",tn )
print("Confusion Matrix")
print(cnf_matrix)


print("############## Usefulness")

clf = svm.OneClassSVM(nu=0.1,kernel="poly",gamma=0.1)
clf.fit(regularTrainSet_converted)
svmoneclass = clf.predict(requirementTest)
actualResults = np.array(usefulness_test)
print("Actual results:")

print("Classification report for SVM One class")
result = classification_report(actualResults,svmoneclass)
print(result)

cnf_matrix = confusion_matrix(novelty_test,svmoneclass)
tp, fn, fp, tn = confusion_matrix(usefulness_test,svmoneclass).ravel()
print("tp:",tp,"fn:",fn, "fp:",fp, "tn:",tn )
print("Confusion Matrix")
print(cnf_matrix)




#########################Novelty
Actual results:
Classification report for SVM One class
             precision    recall  f1-score   support

         -1       0.73      0.75      0.74       372
          1       0.33      0.31      0.32       148

avg / total       0.62      0.63      0.62       520

tp: 280 fn: 92 fp: 102 tn: 46
Confusion Matrix
[[280  92]
 [102  46]]
############## Clarity
Actual results:
Classification report for SVM One class
             precision    recall  f1-score   support

         -1       0.96      0.83      0.89       496
          1       0.07      0.25      0.11        24

avg / total       0.92      0.80      0.85       520

tp: 412 fn: 84 fp: 18 tn: 6
Confusion Matrix
[[308  64]
 [122  26]]
############## Creativity
Actual results:
Classification report for SVM One class
             precision    recall  f1-score   support

         -1       0.86      0.88      0.87       444
          1       0.17      0.14      0.16        76

avg / total       0.7